In [0]:
# These are all the models we'll be using later. Make sure you can import them 
# before procedding further
%matplotlib inline
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import bz2
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import nltk # standard preprocessing
import operator # sorting items in dictionary by value 
# nltk.download() #tokensizers/plunkt/PY3/english.pickle
from math import ceil
import csv


In [0]:
filename = '/content/wikipedia2text-extracted.txt.bz2'

In [0]:
!rm -rf /wikipedia2text-extracted.txt.bz2

In [0]:
!rm -rf /content/wikipedia2text-extracted.txt.bz2

In [28]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [29]:
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""

  with bz2.BZ2File(filename) as f:
    data = []
    file_string = f.read().decode('utf-8')
    file_string = nltk.word_tokenize(file_string)
    data.extend(file_string)
  return data
  
words = read_data(filename)
print('Data size %d' % len(words))
print('Example words (start): ',words[:10])
print('Example words (end): ',words[-10:])

Data size 11631723
Example words (start):  ['Propaganda', 'is', 'a', 'concerted', 'set', 'of', 'messages', 'aimed', 'at', 'influencing']
Example words (end):  ['useless', 'for', 'cultivation', '.', 'and', 'people', 'have', 'sex', 'there', '.']


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
df = pd.read_csv('/content/sample_data/california_housing_test.csv')

In [0]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [0]:
print('Data size %d'% len(words))
print('Example words (start): ', words[:10])
print('Example words (end)', words[-10:])

NameError: ignored

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Read Data with Preprocessing with NLTK

Reads data as it is to a string, convert to lower-case and tokenize it using the nltk library. This code reads data in 1MB portions as processing the full text at once slows down the task and returns a list of words. You will have to download the necessary tokenizer.

In [30]:
def read_data(filename):
          """
              Extract the first file enclosed in a zip file as a list of words
              and pre-processes it using the nltk python library
          """

          with bz2.BZ2File(filename) as f:

            data = []
            file_size = os.stat(filename).st_size
            chunck_size = 1024 * 1024 # reading 1mb at a time as dataset is modaretaly large
            print('Data data ......')
            for i in range(ceil(file_size//chunck_size)+1):
              bytes_to_read = min(chunck_size, file_size - (i * chunck_size))
              file_string = f.read(bytes_to_read).decode('utf-8')
              file_string = file_string.lower()
              # tokenizes a string to words residing a list
              file_string = nltk.word_tokenize(file_string)
              data.extend(file_string)

            return data

words = read_data(filename)
print('Data size %d' % len(words))
print('Example words (start): ', words[:10])
print('Example words (end): ', words[-10:])



Data data ......
Data size 3360286
Example words (start):  ['propaganda', 'is', 'a', 'concerted', 'set', 'of', 'messages', 'aimed', 'at', 'influencing']
Example words (end):  ['favorable', 'long-term', 'outcomes', 'for', 'around', 'half', 'of', 'those', 'diagnosed', 'with']


## Building the Dictionaries

Builds the following. To understand each of theses elements, let us also assume the text, "I like to go school"

* `dictionary` : maps a string word to an ID (e.g. {I:0, like:1, to:2, go:3, school 4})
* `reverse_dictionary`: maps an ID to a string word (e.g. {0:I, 1:like, 2:to, 3:go, 4:school}
* `count`: List of list of (word, frequency) elements (e.g. [(I,1),(like,1),(to,2),(go,1),(school,1)]
* `data` : Contain the string of text we read, where string words are replaced with word IDs (e.g. [0, 1, 2, 3, 2, 4])

It also introduces an additional special token `UNK` to denote rare words to are too rare to make use of.

In [32]:
# We restrict our vocabulary size to 50000
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]
  # Get only the vocabulary_size most common words as the vocabulary
  # All the other words will be replaced with UNK token

  count.extend(collections.Counter(words).most_common(vocabulary_size -1))
  dictionary = dict()

  # Create an ID for each word by giving the current length of the dictionary
  # And adding that item to the dictionary 
  for word, _ in count:
    dictionary[word] = len(dictionary)

  data = list()
  unk_count = 0
  # Traverse through all the text we have and produce a list
  # Where each element correponds to the ID of the word found at that index

  for word in words:
    # If word is in dictionary, use the word ID
    # Else the id of the special token "UNK"
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0 # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)

    # Update the count variable with the UNK resources
    count[0][1] = unk_count

    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    # Make sure the dictionary is of the size of vocabulary
    assert len(dictionary) == vocabulary_size

    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words 





Most common words (+UNK) [['UNK', 0], ('the', 226881), (',', 184013), ('.', 120944), ('of', 116323)]
Sample data [1721]


## Generating Batches of Data for Skim Gram

Generates a batch or target words (batch) and a batch of corresponding context words (labels). It reads 2*window_size+1 words at a time (called a span) and create 2*window_size data points in a single points. The function continue in this manner until batch_size datapoints are created. Everytime we reach the end of the word sequance, we start from beginning. 


In [33]:
data_index = 0

def generate_batch_skip_gram(batch_size, window_size):
  # data_index is updated by 1 everytime we read a data point
  global data_index

  # Two numpy array to hold target words (batch)
  # and context words (labels)
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

  # span defines the total window size, where
  # data we consider at an instance looks as follow.
  # [ skip_window target skip_window]
  span = 2 * window_size + 1

  # The buffer holds the data contained within the span
  buffer = collections.deque(maxlen=span)

  # Fill the buffer and update the data_index 
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)

  # This is the number of context words we sample for a single target word
  num_samples = 2*window_size

  # We break the batch reading into two for loops
  # The inner for loop fills in the batch and labels with
  # num_samples data points using data contained within span
  # The outer for loop repeat this for batch_size//num_samples times
  # to produce a full batch

  for i in range(batch_size//num_samples):
    k=0
    # avoid the target word itself as a prediction
    # fill in batch and label numpy array
    for j in list(range(window_size)) + list(range(window_size + 1, 2 * window_size + 1)):
      batch[i * num_samples + k] = buffer[window_size]
      labels[i * num_samples + k, 0] = buffer[j]
      k += 1

    # Everytime we read num_samples data points.
    # We have created the maximum number of datapoints possible
    # within a single span, so we need to move the span by 1
    # to create a fresh new span
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for window_size in [1, 2]:
  data_index = 0
  batch, labels = generate_batch_skip_gram(batch_size=8, window_size=window_size)
  print('\n with window_size = %d:' % window_size)
  print(' batch:', [reverse_dictionary[bi] for bi in batch])
  print(' labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data: ['propaganda']

 with window_size = 1:
 batch: ['propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda']
 labels: ['propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda']

 with window_size = 2:
 batch: ['propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda']
 labels: ['propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda', 'propaganda']


## Skip-Gram Algorithm
### Defining Hyperparameters

Here we define several hyperparameters including batch_size (amount of samples in a single batch) embedding_size (size of embedding vectors) window_size (context window size).

In [0]:
batch_size = 128 # Data points in a single batch
embedding_size = 128 # Dimension of the embedding vector
window_size = 4 # How many words to consider left and right.

# We pick a random validation set to sample nearest neighbors 
valid_size = 16 # Random set of words to evaluate similarity on.
# We sample valid datapoints randomly from a large window without always being deterministic
valid_window = 50

# When selecting valid examples, we select some of the most frequent words as well as 
# some moderately rare words as well
valid_examples = np.array(random.sample(range(valid_window), valid_size))
valid_examples = np.append(valid_examples,random.sample(range(1000, 1000+valid_window), valid_size),axis=0)

num_sampled = 32 # Number of negative examples to sample


## Defining Inputs and Outputs

Here we define placeholders for feeding in training inputs and outputs (each of size batch_size) and a constant tensor to contain validation examples.

In [0]:
tf.reset_default_graph()

# Training input data (target word IDs)
train_dataset = tf.placeholder(tf.int32, shape=[batch_size])

# Training input label data (context word IDS)
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
# Validation input data, we don't need a placeholder
# as we have already defined the IDs of the word selected
# as validation data
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

# Defining Model Parameters and Other Variables

We now define several TensorFlow variables such as an embedding layer (embeddings) and neural network parameters (softmax_weights and softmax_biases)

In [0]:
# Variables 

# Embedding Layer, contains the word embeddings
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

# Softmax Weights and Biases 
softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size], stddev=0.5 / math.sqrt(embedding_size))
)

softmax_biases = tf.Variable(tf.random_uniform([vocabulary_size], 0.0, 0.01))

#Defining the Model Computations

We first defing a lookup function to fetch the corresponding embedding vectors for a set of given inputs. With that, we define negative sampling loss function tf.nn.sampled_softmax_loss which takes in the embedding vectors and previously defined neural network parameters.

In [0]:
# Model
# Lookup embeddings for a batch of inputs
embed = tf.nn.embedding_lookup(embeddings, train_dataset)

# Compute the softmax loss, using a sample of negative labels each time.
loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(
        weights=softmax_weights, biases=softmax_biases, inputs=embed,
        labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size
    )
)

## Calculating Word Similarities

We calculate the similarity between two given words in terms of the cosine distance. To do this efficiently we use matrix operations to do so, as shown below.

In [0]:
# Compute the similarity between minibatch examples and all embeddings
# We use the cosine distance:
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(
normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))


## Model Parameter Optimizer

We then define a constant learning rate and an optimizer which uses the Adagrad method. Feel free to experiment with other optimizers listed here.

In [41]:
# Optimizer.
optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


#Running the Skip-Gram Algorithm

In [44]:
num_steps = 100001
skip_losses = []

# ConfigProto is a way of providing various configuration settings
# required to execute the graph

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as session:
  # Initialize the variables in the graph
  tf.global_variables_initializer().run()
  print('Initialized')
  average_loss = 0

  # Train the Word2Vec model for num_step iterations
  for step in range(num_steps):

    # Generate a single batch of data
    batch_data, batch_labels = generate_batch_skip_gram(
        batch_size, window_size
    )

    # Populate the feed_dict  and run the optimizer (minimize loss)
    # and compute the loss
    feed_dict = {train_dataset: batch_data, train_labels: batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)

    # Update the average loss variable
    average_loss += 1

    if (step+1) % 2000==0:
      if step > 0:
        average_loss = average_loss / 2000

      skip_losses.append(average_loss)
      # The average loss is an estimate of the loss over the last 2000 batches
      print('Average loss at step %d: %f' % (step+1, average_loss))
      average_loss = 0

    # Evaluating validation set word similarities
    if (step+1) % 10000 == 0:
      sim = similarity.eval()
      # Here we compute the top_k closest words for a given validation word
      # in term of the cosine distance
      # We do this for all the words in the validation set
      # Note: This is an expensive step
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors 
        nearest = (-sim[i, :]).argsort()[1: top_k+1]
        log = 'Nearest to %s' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
      skip_gram_final_embeddings = normalized_embeddings.eval()

  # We will save the word vectors learned and the loss over time
  # as this information is required later for comparisons
  np.save('skip_embeddings', skip_gram_final_embeddings)

  with open('skip_losses.csv', 'wt') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(skip_losses)





Initialized
Average loss at step 2000: 1.000000
Average loss at step 4000: 1.000000
Average loss at step 6000: 1.000000
Average loss at step 8000: 1.000000
Average loss at step 10000: 1.000000
Nearest to ( know, grange, crucially, estates-general, slings, threats, anchorages, arithmetic,
Nearest to city bernicia, fervently, obsolescent, experiential, ting, coded, decisively, braithwaite,
Nearest to it dimetrodon, cansez, chauvet, unelected, handicapped, feduccia, joy, pink,
Nearest to of scandinavian, marpat, painless, mcclory, subsonic, affirms, non-official, 219,
Nearest to its classmate, evert, pointer, cliffs, sähkö, lynching, benfica, husserl,
Nearest to one seconds, 5pm, role-playing, ics, sauvignon, 1516, hamlin, 334,
Nearest to a wrecks, 2004, slaveholders, scorned, rijn, equus, éirinn, 3300,
Nearest to which defiled, depart, linden, dates, alzheimer, complexion, limbo, 7.5,
Nearest to first vagueness, descending, reconciling, ship, nilus, background, spray, logarithms,
Nearest

#Visulizing the Learnings of the Skip-Gram Algorithm

## Finding Only the Words Clustered Together Instead of Sparsely Distributed Words

In [0]:
def find_clustered_embeddings(embeddings, distance_threshold, sample_threshold):
  """
      Find only the closely clustered embeddings.
      This gets rid of more sparsly distributed word embeddings and make the visualization clearer
      This is useful for t-SNE visualization

      distance_threshold: maximum distance between two points to qualify as neighbors
      sample_threshold: number of neighbors required to be considered cluster
  
  """

  # calculate cosine similarity
  cosine_sim = np.dot(embeddings, np.transpose(embeddings))
  norm = np.dot(np.sum(embeddings**2, axis=1).reshape(-1, 1), np.sum(np.transpose(embeddings)**2, axis=0).reshape(1, -1))
  assert cosine_sim.shape == norm.shape
  cosine_sim /= norm

  # make all the diagnol entries zero otherwise this will be picked as highest
  np.fill_diagonal(cosine_sim, -1.0)

  argmax_cos_sim = np.argmax(cosine_sim, axis=1)
  mod_cos_sim = cosine_sim

  # Find the maximums in a loop to count if there are more than n items above threshold
  for _ in range(sample_threshold - 1):
    argmax_cos_sim = np.argmax(cosine_sim, axis=1)
    mod_cos_sim[np.arange(mod_cos_sim.shape[0]), argmax_cos_sim] = -1
  
  max_cosine_sim = np.max(mod_cos_sim, axis=1)

  return np.where(max_cosine_sim > distance_threshold)[0]




## Computing the t-SNE Visualization of Word Embeddings Using Scikit-Learn

In [47]:
num_points = 1000 # We will use a large sample space to build the T-SNE manifold then prune it using cosine similarity

tsne = TSNE(perplexity = 30, n_components=2, init='pca', n_iter=5000)

print('Fitting embeddings to T-SNE. This can take some time .......')
# get the T-SNE manifold
selected_embeddings = skip_gram_final_embeddings[:num_points, :]
two_d_embeddings = tsne.fit_transform(selected_embeddings)

print('Pruning the T-SNE embeddings')
# Prune the embeddings by getting ones only more than n-many sample above the similarity threshold
# this ultraclustters the visualization
selected_ids = find_clustered_embeddings(selected_embeddings, .25, 10)
two_d_embeddings = two_d_embeddings[selected_ids, :]

print('Out of ', num_points, ' samples ', selected_ids.shape[0], ' samples were selected by pruning ')

Fitting embeddings to T-SNE. This can take some time .......
Pruning the T-SNE embeddings
Out of  1000  samples  0  samples were selected by pruning 


## Plotting the t-SNE Results with Matplotlib

In [52]:
def plot(embeddings, labels):

  n_clusters = 20 # number of clusters
  # automatically build a discreate set of colors, each of cluster
  cmap = pylab.get_cmap("Spectral")
  label_colors = [cmap(float(i) /n_clusters) for i in range(n_clusters)]

  assert embeddings.shape[0] >= len(labels), 'More label than embeddings'

  # Define K-Means 
  kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=0).fit(embeddings)
  kmeans_labels = kmeans.label_


  pylabe.figure(figsize=(15, 15)) # in inches

  # Plot all the embeddings and their corresponding words
  for i, (label, klabel) in enumerate(zip(labels, kmeans_labels)):
    x, y = embeddings[i, :]
    pylab.scatter(x, y, c=label_colors[klabel])

    pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                   ha='right', va='bottom', fontsize=10)
    
  # Use for saving the figure if needed
  # pylab.savefig('word_embeddings.png')
  pylab.show()

words = [reverse_dictionary[i] for i in selected_ids]
plot(two_d_embeddings, words)

ValueError: ignored